In [1]:
import os
import pandas as pd
import gc
import lightgbm as lgb
import datetime

PROJECT_ROOT = os.path.join(os.getcwd(), '..')
DATA_DIR = os.path.join(PROJECT_ROOT,'data')
MODEL_PATH = os.path.join(PROJECT_ROOT,'model')

In [2]:
def train_and_predict(merge, X_pred):
    X = merge.drop(['click_time', 'attributed_time', 'is_attributed'], axis=1).values
    y = merge['is_attributed'].values
    categorical_features = ['ip','app','os','channel','device', 'day']
    predictors = list(set(merge.columns) - set(['attributed_time', 'click_time', 'is_attributed']))

    lgbtrain = lgb.Dataset(X, label=y,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
        'metric':'auc',     

        'learning_rate': 0.15,
        'num_leaves': 7,  # 2^max_depth - 1
        'max_depth': 3,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        # 'scale_pos_weight':99
    }

    evals_results = {}
    num_boost_round = 250
    early_stopping_rounds = 30

    booster = lgb.train(
         lgb_params, 
         lgbtrain, 
         valid_sets=[lgbtrain], 
         valid_names=['train'], 
         evals_result=evals_results, 
         num_boost_round=num_boost_round,
         early_stopping_rounds=early_stopping_rounds,
         verbose_eval=1
    )
    
    # predict test data
    y_prob = booster.predict(X_pred)
    return y_prob

In [3]:
def preprocess(df):
    df['day'] = df.click_time.dt.day
    df['hour'] = df.click_time.dt.hour
    df['minute'] = df.click_time.dt.minute
    df['second'] = df.click_time.dt.second
    
    return df

In [4]:
def get_now():
    now = datetime.datetime.now()
    return '{0:%Y-%m-%d %H:%M:%S}'.format(now)

In [5]:
def predict(booster):
    
    reader = pd.read_csv(os.path.join(DATA_DIR,'test.csv'), parse_dates=['click_time'], chunksize=100000)
    for i, test in enumerate(reader):
        print('[{}]Start:Preprocessing Data:Size:{}'.format(get_now(), len(test)))
        test = preprocess(test)
        print('[{}]Finish:Preprocessing Data:Size:{}'.format(get_now(), len(test)))
        
        print('[{}]Start:Predicting Data'.format(get_now()))
        X = test.drop(['click_id','click_time'], axis=1)
        y_prob = booster.predict(X.values)
        #y_class_one = [i[1] for i in y_prob]
        print('[{}]Finish:Predicting Data'.format(get_now()))
       
        print('[{}]Start:output Data'.format(get_now()))
        y = pd.DataFrame({
                                        'click_id' : test['click_id'],
                                        'is_attributed' : y_prob
                                        })
        
        output = os.path.join(DATA_DIR, 'submission_lgb.csv')
        if i == 0:
            if os.path.isfile(output):
                os.remove(output)
            header = True
        else:
            header = False
        y.to_csv(output, index=False, header=header, mode='a')
        print('[{}]Finish:output Data'.format(get_now()))
    
    print('[{}]Finish:All Process'.format(get_now()))

In [ ]:
print('[{}]Start:read positive'.format(get_now()))
positive = pd.read_csv(os.path.join(DATA_DIR, 'train_positive.csv'), parse_dates=['click_time'])
print('[{}]Start:read negative'.format(get_now()))
negative = pd.read_csv(os.path.join(DATA_DIR, 'train_negative.csv'), parse_dates=['click_time'])
print('[{}]Start:read test'.format(get_now()))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'), parse_dates=['click_time'])
test = preprocess(test)
X = test.drop(['click_id','click_time'], axis=1)
click_id = test['click_id']
del test
gc.collect()
X_pred = X.values

print('[{}]Finished:All data preparing'.format(get_now()))

y_probs = []
for i in range(200):
    print('[{}]Start: process:{}'.format(get_now(), i))
    negative_sampled = negative.sample(500000)
    merge = pd.concat([positive, negative_sampled])
    merge = preprocess(merge)
    print('[{}]Start: process:{}:training'.format(get_now(), i))
    y_probs.append(train_and_predict(merge, X_pred))
    print('[{}]Finished: process:{}:training'.format(get_now(), i))

del negative
gc.collect()

[2018-04-24 01:31:48]Start:read positive
[2018-04-24 01:31:49]Start:read negative
[2018-04-24 01:39:48]Start:read test
[2018-04-24 01:40:39]Finished:All data preparing
[2018-04-24 01:40:39]Start: process:0
[2018-04-24 01:41:48]Start: process:0:training


/Users/koumei/anaconda/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	train's auc: 0.937253
Training until validation scores don't improve for 30 rounds.
[2]	train's auc: 0.940282
[3]	train's auc: 0.943017
[4]	train's auc: 0.943316
[5]	train's auc: 0.947896
[6]	train's auc: 0.947176
[7]	train's auc: 0.947553
[8]	train's auc: 0.947992
[9]	train's auc: 0.948615
[10]	train's auc: 0.949095
[11]	train's auc: 0.949252
[12]	train's auc: 0.949598
[13]	train's auc: 0.949914
[14]	train's auc: 0.950084
[15]	train's auc: 0.950508
[16]	train's auc: 0.95081
[17]	train's auc: 0.951121
[18]	train's auc: 0.95115
[19]	train's auc: 0.95127
[20]	train's auc: 0.951779
[21]	train's auc: 0.952157
[22]	train's auc: 0.952517
[23]	train's auc: 0.953041
[24]	train's auc: 0.953304
[25]	train's auc: 0.953639
[26]	train's auc: 0.954082
[27]	train's auc: 0.954641
[28]	train's auc: 0.954627
[29]	train's auc: 0.954933
[30]	train's auc: 0.95507
[31]	train's auc: 0.955228
[32]	train's auc: 0.955345
[33]	train's auc: 0.955379
[34]	train's auc: 0.95572
[35]	train's auc: 0.955854
[36]	tr

[39]	train's auc: 0.956224
[40]	train's auc: 0.956323
[41]	train's auc: 0.956348
[42]	train's auc: 0.956378
[43]	train's auc: 0.956565
[44]	train's auc: 0.956729
[45]	train's auc: 0.957104
[46]	train's auc: 0.957237
[47]	train's auc: 0.95751
[48]	train's auc: 0.957599
[49]	train's auc: 0.957703
[50]	train's auc: 0.958145
[51]	train's auc: 0.958241
[52]	train's auc: 0.958344
[53]	train's auc: 0.95839
[54]	train's auc: 0.958443
[55]	train's auc: 0.958486
[56]	train's auc: 0.958682
[57]	train's auc: 0.95871
[58]	train's auc: 0.958797
[59]	train's auc: 0.958878
[60]	train's auc: 0.958915
[61]	train's auc: 0.958955
[62]	train's auc: 0.959066
[63]	train's auc: 0.959112
[64]	train's auc: 0.959159
[65]	train's auc: 0.959166
[66]	train's auc: 0.959646
[67]	train's auc: 0.959689
[68]	train's auc: 0.95992
[69]	train's auc: 0.959944
[70]	train's auc: 0.959982
[71]	train's auc: 0.960016
[72]	train's auc: 0.96005
[73]	train's auc: 0.960086
[74]	train's auc: 0.960118
[75]	train's auc: 0.960153
[76]	t

[80]	train's auc: 0.960226
[81]	train's auc: 0.960286
[82]	train's auc: 0.960313
[83]	train's auc: 0.960342
[84]	train's auc: 0.960371
[85]	train's auc: 0.960487
[86]	train's auc: 0.960511
[87]	train's auc: 0.960554
[88]	train's auc: 0.960581
[89]	train's auc: 0.960648
[90]	train's auc: 0.960677
[91]	train's auc: 0.960746
[92]	train's auc: 0.960774
[93]	train's auc: 0.960803
[94]	train's auc: 0.96083
[95]	train's auc: 0.960841
[96]	train's auc: 0.960867
[97]	train's auc: 0.960893
[98]	train's auc: 0.960919
[99]	train's auc: 0.960942
[100]	train's auc: 0.960968
[101]	train's auc: 0.960991
[102]	train's auc: 0.961014
[103]	train's auc: 0.961033
[104]	train's auc: 0.961055
[105]	train's auc: 0.961078
[106]	train's auc: 0.9611
[107]	train's auc: 0.961106
[108]	train's auc: 0.961125
[109]	train's auc: 0.961151
[110]	train's auc: 0.961148
[111]	train's auc: 0.961272
[112]	train's auc: 0.961291
[113]	train's auc: 0.961315
[114]	train's auc: 0.961333
[115]	train's auc: 0.961336
[116]	train's a

...

[90]	train's auc: 0.96105
[91]	train's auc: 0.96106
[92]	train's auc: 0.961092
[93]	train's auc: 0.961115
[94]	train's auc: 0.961138
[95]	train's auc: 0.961163
[96]	train's auc: 0.961185
[97]	train's auc: 0.961209
[98]	train's auc: 0.961231
[99]	train's auc: 0.961255
[100]	train's auc: 0.961276
[101]	train's auc: 0.961294
[102]	train's auc: 0.961316
[103]	train's auc: 0.961336
[104]	train's auc: 0.961365
[105]	train's auc: 0.961389
[106]	train's auc: 0.961412
[107]	train's auc: 0.96143
[108]	train's auc: 0.961452
[109]	train's auc: 0.961474
[110]	train's auc: 0.961492
[111]	train's auc: 0.961508
[112]	train's auc: 0.96153
[113]	train's auc: 0.96155
[114]	train's auc: 0.961569
[115]	train's auc: 0.961601
[116]	train's auc: 0.961616
[117]	train's auc: 0.961635
[118]	train's auc: 0.961651
[119]	train's auc: 0.96167
[120]	train's auc: 0.961695
[121]	train's auc: 0.961712
[122]	train's auc: 0.961737
[123]	train's auc: 0.961762
[124]	train's auc: 0.961886
[125]	train's auc: 0.961906
[126]	tr

[129]	train's auc: 0.962405
[130]	train's auc: 0.96242
[131]	train's auc: 0.962439
[132]	train's auc: 0.962457
[133]	train's auc: 0.962475
[134]	train's auc: 0.962487
[135]	train's auc: 0.9625
[136]	train's auc: 0.962515
[137]	train's auc: 0.962532
[138]	train's auc: 0.962549
[139]	train's auc: 0.962565
[140]	train's auc: 0.962581
[141]	train's auc: 0.962596
[142]	train's auc: 0.96261
[143]	train's auc: 0.962625
[144]	train's auc: 0.962696
[145]	train's auc: 0.962711
[146]	train's auc: 0.962723
[147]	train's auc: 0.962738
[148]	train's auc: 0.962753
[149]	train's auc: 0.96277
[150]	train's auc: 0.962784
[151]	train's auc: 0.9628
[152]	train's auc: 0.962812
[153]	train's auc: 0.962826
[154]	train's auc: 0.962825
[155]	train's auc: 0.962864
[156]	train's auc: 0.962879
[157]	train's auc: 0.962892
[158]	train's auc: 0.962906
[159]	train's auc: 0.962916
[160]	train's auc: 0.962928
[161]	train's auc: 0.962942
[162]	train's auc: 0.963118
[163]	train's auc: 0.96313
[164]	train's auc: 0.963143


[167]	train's auc: 0.96296
[168]	train's auc: 0.962974
[169]	train's auc: 0.962985
[170]	train's auc: 0.962997
[171]	train's auc: 0.963008
[172]	train's auc: 0.963023
[173]	train's auc: 0.963035
[174]	train's auc: 0.963047
[175]	train's auc: 0.963057
[176]	train's auc: 0.96307
[177]	train's auc: 0.963083
[178]	train's auc: 0.9631
[179]	train's auc: 0.963114
[180]	train's auc: 0.963128
[181]	train's auc: 0.96314
[182]	train's auc: 0.963146
[183]	train's auc: 0.963201
[184]	train's auc: 0.963214
[185]	train's auc: 0.963226
[186]	train's auc: 0.963238
[187]	train's auc: 0.963252
[188]	train's auc: 0.963263
[189]	train's auc: 0.963274
[190]	train's auc: 0.963283
[191]	train's auc: 0.963294
[192]	train's auc: 0.963341
[193]	train's auc: 0.963353
[194]	train's auc: 0.963361
[195]	train's auc: 0.963472
[196]	train's auc: 0.963509
[197]	train's auc: 0.96352
[198]	train's auc: 0.963531
[199]	train's auc: 0.963543
[200]	train's auc: 0.963587
[201]	train's auc: 0.963596
[202]	train's auc: 0.96360

[205]	train's auc: 0.963958
[206]	train's auc: 0.963972
[207]	train's auc: 0.963982
[208]	train's auc: 0.963991
[209]	train's auc: 0.964002
[210]	train's auc: 0.964011
[211]	train's auc: 0.96402
[212]	train's auc: 0.964029
[213]	train's auc: 0.96404
[214]	train's auc: 0.964051
[215]	train's auc: 0.964092
[216]	train's auc: 0.9641
[217]	train's auc: 0.964111
[218]	train's auc: 0.96412
[219]	train's auc: 0.964133
[220]	train's auc: 0.964144
[221]	train's auc: 0.964152
[222]	train's auc: 0.96416
[223]	train's auc: 0.964169
[224]	train's auc: 0.964177
[225]	train's auc: 0.964189
[226]	train's auc: 0.964198
[227]	train's auc: 0.964207
[228]	train's auc: 0.964216
[229]	train's auc: 0.964226
[230]	train's auc: 0.964234
[231]	train's auc: 0.964244
[232]	train's auc: 0.964253
[233]	train's auc: 0.964258
[234]	train's auc: 0.964292
[235]	train's auc: 0.9643
[236]	train's auc: 0.964308
[237]	train's auc: 0.964339
[238]	train's auc: 0.964427
[239]	train's auc: 0.964434
[240]	train's auc: 0.964442


[243]	train's auc: 0.964434
[244]	train's auc: 0.964443
[245]	train's auc: 0.964453
[246]	train's auc: 0.964461
[247]	train's auc: 0.96447
[248]	train's auc: 0.964479
[249]	train's auc: 0.964513
[250]	train's auc: 0.964523
Did not meet early stopping. Best iteration is:
[250]	train's auc: 0.964523
[2018-04-24 07:33:22]Finished: process:67:training
[2018-04-24 07:33:22]Start: process:68
[2018-04-24 07:34:35]Start: process:68:training
[1]	train's auc: 0.93701
Training until validation scores don't improve for 30 rounds.
[2]	train's auc: 0.940123
[3]	train's auc: 0.942452
[4]	train's auc: 0.942503
[5]	train's auc: 0.947709
[6]	train's auc: 0.946924
[7]	train's auc: 0.947318
[8]	train's auc: 0.948197
[9]	train's auc: 0.949107
[10]	train's auc: 0.9491
[11]	train's auc: 0.949135
[12]	train's auc: 0.949541
[13]	train's auc: 0.949866
[14]	train's auc: 0.950153
[15]	train's auc: 0.950537
[16]	train's auc: 0.9507
[17]	train's auc: 0.950909
[18]	train's auc: 0.951135
[19]	train's auc: 0.95137
[20